# Savannah Bikeways
- GIS File: https://data-sagis.opendata.arcgis.com/datasets/SAGIS::bikeways/about
- Plan: https://www.thempc.org/Core/Bpp#gsc.tab=0
---
This notebook is for processing and cleaning the Savannah bikeways data. This layer will be used for assigning improvements the Savannah network so that they can be assessed with BikewaySim

In [ ]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import json

config = json.load((Path.cwd().parent / 'config.json').open('rb'))
network_fp = Path(config['project_directory']) / "Network"
osmdwnld_fp = Path(config['project_directory']) / "OSM_Download"
elevation_fp = Path(config['project_directory']) / "Elevation"
cyclinginfra_fp = Path(config['project_directory']) / "Cycling_Infra_Dating"
calibration_fp = Path(config['project_directory']) / "Calibration"

In [ ]:
bikeways = gpd.read_file(Path.home()/"Documents/BikewaySim/RAW/SAGIS/Bikeways.geojson")
bikeways.columns

In [ ]:
bikeways.crs

In [ ]:
drop_cols = ['OBJECTID','RtA', 'RtB', 'RtC', 'SegA', 'SegB',
            'SegC','Route_Nums','History', 'Avg_Daily_Traffic', 'Directional_Factor',
            'Peak_to_Daily_Factor', 'Peak_Hr_Factor', 'Peak_15_Min',
            'Total_Directional_Thru_Lanes', 'Speed_Posted', 'Speed_Effective',
            'Heavy_Veh_Percent', 'Pave_Rating_FHWA', 'Total_Width_Outlane_Shldr',
            'Occ_On_St_Park', 'Width_Pave_Outstripe_to_Edge', 'Width_On_St_Park',
            'Width_Function_of_Vol', 'St_Undivided_Unstriped',
            'Ln_Width_Avg_Effective', 'LOS_Score', 'LOS_Category', 'EXIST_CD',
       'GlobalID', 'SHAPESTLength','Signed_Rt']
bikeways.drop(columns=drop_cols,inplace=True)

Existing or EXIST_CD (Existing Type) field:
- BL or 101 = Bike Lane
- CT or 102 = Cycle Track
- BP or 103 = Bike Path (i.e. Shared Use Path)
- PS or 104 = Paved Shoulder
- Narrow PS or 105 = Narrow Paved Shoulder
- SL or 106 = Shared Lane
- WCL or 107 = Wide Curb Lane
- 108 = Unopened

In [ ]:
data_dictionary = {
    'BL': 'Bike Lane',
    101: 'Bike Lane',
    'CT': 'Cycle Track',
    102: 'Cycle Track',
    'BP': 'Shared Use Path',#'Bike Path (i.e. Share Use Path)',
    103: 'Shared Use Path',#'Bike Path (i.e. Share Use Path)',
    'PS': 'Paved Shoulder',
    104: 'Paved Shoulder',
    'NarrowPS': 'Narrow Paved Shoulder',
    105: 'Narrow Paved Shoulder',
    'SL': 'Shared Lane',
    106: 'Shared Lane',
    'WCL': 'Wide Curb Lane',
    107: 'Wide Curb Lane',
    108: 'Unopened'
}

#bikeways['EXIST_CD'] = bikeways['EXIST_CD'].map(data_dictionary)
bikeways['Existing'] = bikeways['Existing'].map(data_dictionary)

Status_by_Type (Status of Plan) field:
- 0 = Existing Bike Lane
- 1 = Existing Shared Use Path
- 2 = Existing Paved Shoulder
- 3 = Existing Shared Lane
- 4 = Existing Wide Curb Lane
- 5 = Recommended Bike Lane
- 6 = Recommended Shared Use Path
- 7 = Recommended Paved Shoulder
- 8 = Recommended Wide Curb Lane
- 9 = Existing Cycle Track
- 10 = Recommended Cycle Track
- 11 = Existing Narrow Paved Shoulder
- 12 = Recommended Narrow Paved Shoulder
- 13 = Recommended Shared Lane

In [ ]:
data_dictionary = {
    0 : "Existing Bike Lane",
    1 : "Existing Shared Use Path",
    2 : "Existing Paved Shoulder",
    3 : "Existing Shared Lane",
    4 : "Existing Wide Curb Lane",
    5 : "Recommended Bike Lane",
    6 : "Recommended Shared Use Path",
    7 : "Recommended Paved Shoulder",
    8 : "Recommended Wide Curb Lane",
    9 : "Existing Cycle Track",
    10 : "Recommended Cycle Track",
    11 : "Existing Narrow Paved Shoulder",
    12 : "Recommended Narrow Paved Shoulder",
    13 : "Recommended Shared Lane"
}

In [ ]:
bikeways['Status_Type'] = bikeways['Status_Type'].map(data_dictionary)

In [ ]:
improvements = bikeways[bikeways['Status_Type'].str.contains('Recommended')].copy()
improvements.loc[:,'Recommended'] = improvements['Status_Type'].apply(lambda x: x.split('Recommended ')[-1])

drop_no_improvement = improvements['Existing'] != improvements['Recommended']
improvements = improvements[drop_no_improvement]

drop_facils = ['Shared Lane','Paved Shoulder','Narrow Paved Shoulder','Wide Curb Lane']
improvements = improvements[improvements['Recommended'].isin(drop_facils)==False]

In [ ]:
improvements[['Existing','Recommended']].value_counts(dropna=False)

In [ ]:
m = improvements.explore('Status_Type',tiles='CartoDBpositron')
m.save(cyclinginfra_fp/'savannah_improvements.html')

In [ ]:
cyclinginfra_fp

In [ ]:



paved_shoulders_and_curb_lanes = improvements['Status_Type'].str.contains('Paved Shoulder') | improvements['Status_Type'].str.contains('Wide Curb Lane')
sharrow


improvements = improvements[paved_shoulders_and_curb_lanes==False]






In [ ]:
#take out paved shoulders and wide curb lanes
#just focus on cycle tracks, bike lanes, and sups

In [ ]:
bikeways[['Existing','Status_Type']]